#  Balancing Privacy and Accuracy in Machine Learning Models with Differential Privacy

## 1 . Introduction & Problem Statement

This project explores how **Differential Privacy Stochastic Gradient Descent (DP-SGD)** and **Model Agnostic Private Learning (MAPL)** impact machine learning models. Specifically, we examine whether DP techniques interfere with model accuracy and the evolution of machine learning.

## 2 . Setup - Install Required Packages

Run the code below if there are missing packages:

In [ ]:
# %pip install matplotlib
# %pip install seaborn
# %pip install pandas
# %pip install scikit-learn==1.3.2
# %pip install ucimlrepo
# %pip install imblearn
# %pip install tabulate
# %pip install boruta
# %pip install torch
# %pip install opacus
# %pip install scipy
# %pip install numpy
# %pip install imbalanced-learn==0.11.0

import importlib.util

required = {
    "matplotlib",
    "seaborn",
    "pandas",
    "ucimlrepo",
    "tabulate",
    "boruta",
    "torch",
    "opacus",
    "scipy",
    "numpy==1.26.4",
}

installed = {pkg for pkg in required if importlib.util.find_spec(pkg.split('=')[0]) is not None}
# Special handling for scikit-learn as it may be installed under the name 'sklearn'
if importlib.util.find_spec("sklearn") is not None:
    installed.add("scikit-learn")
# Special handling for imbalanced-learn as it is installed under the name 'imblearn'
if importlib.util.find_spec("imblearn") is not None:
    installed.add("imbalanced-learn")
missing = required - installed

if missing:
    print(f"The following libraries are missing: {', '.join(missing)}")
    print("Attempting to install missing libraries...")
    %pip install {" ".join(missing)}
    print("Libraries installed successfully.")
else:
    print("All required libraries are installed.")

In [ ]:
import os
import pickle

def save_file(data, file_path, params={}):
    if not os.path.exists(os.path.dirname(file_path)):
        print(f"Creating directory: {os.path.dirname(file_path)}")
        os.makedirs(os.path.dirname(file_path))
    if file_path.endswith('.csv'):
        data.to_csv(file_path, index=False, **params)
    elif file_path.endswith('.png') or file_path.endswith('.jpg'):
        data.savefig(file_path)
    elif file_path.endswith('.pkl'):
        with open(file_path, 'wb') as f:
            pickle.dump(data, f)
    
results_gInfo_dir = "results/generalInfo"
results_fsData_dir = "results/featureSelectionData"
results_models_dir = "results/models"
models_dir = "models"

## 3 . Data Exploration & Understanding

We are importing the relevant libraries and getting the data:

In [ ]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ucimlrepo import fetch_ucirepo #to retrieve the dataset

# Getting the data
dataset_id = 891  # our chosen dataset
dataset_name = "dataset.csv"
downloaded = os.path.isfile(dataset_name)
df =  pd.read_csv(dataset_name) if downloaded else fetch_ucirepo(id=dataset_id).data.original

# # Previous way to retrieve the dataset (for documentation)
# url = 'https://archive.ics.uci.edu/static/public/891/data.csv'
# df = pd.read_csv(url)

# Save the dataset to a CSV file
if not downloaded:
    print("Dataset not found locally. Downloading...")
    save_file(df, "./" + dataset_name)

# Set display options
pd.set_option('display.max_columns', None)


We have observed that the dataset contains 253,680 records with 23 features:

In [ ]:
# Checking basic Dataset Information
print("Dataset Shape:", df.shape)
print("\nFirst 5 rows:")
df.head()

We have observed that all features are stored as integer data types:

The following table shows the features and their descriptions:

| Variable Name     | Role    | Type    | Description                                                                 |
|-------------------|---------|---------|-----------------------------------------------------------------------------|
| ID                | ID      | Integer | Patient ID                                                                  |
| Diabetes_binary   | Target  | Binary  | 0 = no diabetes 1 = prediabetes or diabetes                                 |
| HighBP            | Feature | Binary  | 0 = no high BP 1 = high BP                                                  |
| HighChol          | Feature | Binary  | 0 = no high cholesterol 1 = high cholesterol                                |
| CholCheck         | Feature | Binary  | 0 = no cholesterol check in 5 years 1 = yes cholesterol check in 5 years    |
| BMI               | Feature | Integer | Body Mass Index                                                             |
| Smoker            | Feature | Binary  | Have you smoked at least 100 cigarettes in your entire life? [Note: 5 packs = 100 cigarettes] 0 = no 1 = yes |
| Stroke            | Feature | Binary  | (Ever told) you had a stroke. 0 = no 1 = yes                                |
| HeartDiseaseorAttack | Feature | Binary | coronary heart disease (CHD) or myocardial infarction (MI) 0 = no 1 = yes  |
| PhysActivity      | Feature | Binary  | physical activity in past 30 days - not including job 0 = no 1 = yes        |
| Fruits            | Feature | Binary  | Consume Fruit 1 or more times per day 0 = no 1 = yes                        |
| Veggies           | Feature | Binary  | Consume Vegetables 1 or more times per day 0 = no 1 = yes                   |
| HvyAlcoholConsump | Feature | Binary  | Heavy drinkers (adult men having more than 14 drinks per week and adult women having more than 7 drinks per week) 0 = no 1 = yes |
| AnyHealthcare     | Feature | Binary  | Have any kind of health care coverage, including health insurance, prepaid plans such as HMO, etc. 0 = no 1 = yes |
| NoDocbcCost       | Feature | Binary  | Was there a time in the past 12 months when you needed to see a doctor but could not because of cost? 0 = no 1 = yes |
| GenHlth           | Feature | Integer | General health: 1 = excellent 2 = very good 3 = good 4 = fair 5 = poor |
| MentHlth          | Feature | Integer | For how many days during the past 30 days was your mental health not good? scale 1-30 days |
| PhysHlth          | Feature | Integer | For how many days during the past 30 days was your physical health not good? scale 1-30 days |
| DiffWalk          | Feature | Binary  | Do you have serious difficulty walking or climbing stairs? 0 = no 1 = yes   |
| Sex               | Feature | Binary  | 0 = female 1 = male                                                    |
| Age               | Feature | Integer | 13 levels, 1 = 18-24, 9 = 60-64, 13 = 80 or older              |
| Education         | Feature | Integer | 6 levels, 1 = Never attended school or only kindergarten, 2 = Grades 1 through 8, 3 = Grades 9 through 11, 4 = Grade 12 or GED, 5 = College 1 year to 3 years, 6 = College 4 years or more |
| Income            | Feature | Integer | 8 levels, 1 = less than $10,000, 5 = less than $35,000, 8 = $75,000 or more |


In [ ]:
# Checking data types
print("\nData Types:")
print(df.dtypes)

We have observed basic statistics for the features:

In [ ]:
# Checking basic statistics
print("\nBasic Statistics:")
print(df.describe())

# Save basic statistics to a CSV file
save_file(df.describe(), os.path.join(results_gInfo_dir, "basic_statistics.csv"))

We have observed that there are no missing values (as stated in the dataset description):

In [ ]:
# Checking for missing values
print("\nMissing values per column:")
df.isnull().sum()

We have observed that the dataset only has 13.93% records with diabetes. We will address this imbalance later during data cleaning and preprocessing:

In [ ]:
# Checking distribution of dataset
diabetes_counts = df['Diabetes_binary'].value_counts()
print("Distribution of target variable:")
print(diabetes_counts)
print(f"Percentage of records with diabetes: {diabetes_counts[1]/len(df)*100:.2f}%")
# Plot the distribution of the target variable
plt.figure(figsize=(8, 6))
diabetes_counts.plot(kind='bar', color=['skyblue', 'orange'])
plt.title("Distribution of Target Variable (Diabetes_binary)")
plt.xlabel("Diabetes_binary")
plt.ylabel("Count")
plt.xticks(ticks=[0, 1], labels=[f"No Diabetes ({diabetes_counts[0]:,})", f"Diabetes ({diabetes_counts[1]:,})"], rotation=0)
plt.tight_layout()

# Save the plot
plot_path = os.path.join(results_gInfo_dir, "imbalanced_diabetes_distribution.png")
save_file(plt, plot_path)
plt.show()

## 4 . Preprocessing

Usually in preprocessing we would handle missing values, encoding binary data (True -> 1), encoding categorical data, feature scaling. However, since the dataset is already clean and preprocessed, we will only address the class imbalance issue.
In the balanced datased, we have reduced the number of records with diabetes to match the number of records without diabetes, so that the dataset is balanced.

### 4.1 Class Balancing

In [ ]:
num_with_diabetes = df['Diabetes_binary'].value_counts()[1]

print("\nOriginal Dataset:")
print(f"- Total samples in the original dataset: {len(df)}")
print(f"- Samples with diabetes (class 1): {num_with_diabetes}")
print(f"- Samples without diabetes (class 0): {df['Diabetes_binary'].value_counts()[0]}")

df_no_diabetes = df[df['Diabetes_binary'] == 0].sample(n=num_with_diabetes, random_state=42)
df_with_diabetes = df[df['Diabetes_binary'] == 1]

df_balanced = pd.concat([df_no_diabetes, df_with_diabetes])
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)
df_balanced_noTarget = df_balanced.drop(columns=["Diabetes_binary", "ID"])
y_balanced = df_balanced["Diabetes_binary"]

print("\nBalanced Dataset:")
print(f"- Total samples in the balanced dataset: {len(df_balanced)}")
print(df_balanced['Diabetes_binary'].value_counts())

print(f"- Samples without diabetes (class 0): {df_balanced['Diabetes_binary'].value_counts()[0]}")
print(f"- Samples with diabetes (class 1): {df_balanced['Diabetes_binary'].value_counts()[1]}")

# Plot the distribution of the balanced dataset
plt.figure(figsize=(8, 6))
df_balanced['Diabetes_binary'].value_counts().plot(kind='bar', color=['skyblue', 'orange'])
plt.title("Distribution of Target Variable (Balanced Dataset)")
plt.xlabel("Diabetes_binary")
plt.ylabel("Count")
plt.xticks(ticks=[0, 1], labels=["No Diabetes", "Diabetes"], rotation=0)
plt.tight_layout()

# Save the plot
plot_path_balanced = os.path.join(results_gInfo_dir, "balanced_diabetes_distribution.png")
save_file(plt, plot_path_balanced)
plt.show()


### 4.2 Data Splitting Strategy 

Before training our models we have to split the data into a training set and a test set. We have chosen a 80/20 split, which ensures we have enough training data for the model to learn patterns and enough data for performance evaluation. We have used stratified sampling to ensure that both the training set and test set includes a 50/50 split of records having diabetes or not.

In [ ]:
from sklearn.model_selection import train_test_split


X_balanced = df_balanced.drop('Diabetes_binary', axis=1) # drop the target column
y_balanced = df_balanced['Diabetes_binary']

X_imbalanced = df.drop('Diabetes_binary', axis=1) #drop the target column
y_imbalanced = df['Diabetes_binary']

# split into train (70%) and temp (30%)
X_train_bal, X_temp_bal, y_train_bal, y_temp_bal = train_test_split(
    X_balanced, y_balanced, test_size=0.3, random_state=42, stratify=y_balanced
)

# split temp set into validation (10%) and test (20%)
X_val_bal, X_test_bal, y_val_bal, y_test_bal = train_test_split(
    X_temp_bal, y_temp_bal, test_size=2/3, random_state=42, stratify=y_temp_bal
)

# Imbalanced dataset
X_train_imb, X_temp_imb, y_train_imb, y_temp_imb = train_test_split(
    X_imbalanced, y_imbalanced, test_size=0.3, random_state=42, stratify=y_imbalanced
)

X_val_imb, X_test_imb, y_val_imb, y_test_imb = train_test_split(
    X_temp_imb, y_temp_imb, test_size=2/3, random_state=42, stratify=y_temp_imb
)

print(f"Balanced Training set: {X_train_bal.shape[0]} samples")
print(f"Balanced Validation set: {X_val_bal.shape[0]} samples")
print(f"Balanced Testing set: {X_test_bal.shape[0]} samples")

print("\nClass distribution in balanced training set:")
print(y_train_bal.value_counts(normalize=True))

print("\nClass distribution in balanced validation set:")
print(y_val_bal.value_counts(normalize=True))

print("\nClass distribution in balanced testing set:")
print(y_test_bal.value_counts(normalize=True))

print(f"\nImbalanced Training set: {X_train_imb.shape[0]} samples")
print(f"Imbalanced Validation set: {X_val_imb.shape[0]} samples")
print(f"Imbalanced Testing set: {X_test_imb.shape[0]} samples")

print("\nClass distribution in imbalanced training set:")
print(y_train_imb.value_counts(normalize=True))

print("\nClass distribution in imbalanced validation set:")
print(y_val_imb.value_counts(normalize=True))

print("\nClass distribution in imbalanced testing set:")
print(y_test_imb.value_counts(normalize=True))

### 4.3 Feature Selection

This might be a defense against the attacks, because the less features we have, the less information we leak.

We need to explain which features we have chosen based on the data exploration above and the following feature selection methods. We can use variance threshold, correlation matrix, kbest, rfe, boruta...

['HighBP', 'HighChol', 'BMI', 'GenHlth', 'Age'] ok with V.T. and C.M.

Might want to use more features to try the attacks

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import (
    RFE,
    SelectKBest,
    VarianceThreshold,
    mutual_info_classif,
)
from boruta import BorutaPy
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score, classification_report

rf_model = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)

def save_results(results, file_path, columns):
    results_df = pd.DataFrame(results, columns=columns)
    if not os.path.exists(os.path.dirname(file_path)):
        os.makedirs(os.path.dirname(file_path))
    results_df.to_csv(file_path, index=False)

def calculate_metrics(y_test, y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average="weighted")
    recall = recall_score(y_test, y_pred, average="weighted")
    f1 = f1_score(y_test, y_pred, average="weighted")
    return (accuracy, precision, recall, f1)

def getVariances(threshold, data):
    var_threshold = VarianceThreshold(threshold=threshold)
    var_threshold.fit(data)
    variances = var_threshold.variances_
    # Create a DataFrame for variances
    variances_df = pd.DataFrame(variances, index=data.columns, columns=["Variance"])
    # Sort variances in descending order
    variances_df = variances_df.sort_values(by="Variance", ascending=False)
    # Features with variance >= threshold
    features_high_variance = variances_df[variances_df["Variance"] >= threshold]
    # Features with variance < threshold
    features_low_variance = variances_df[variances_df["Variance"] < threshold]
    return variances_df, features_high_variance, features_low_variance

def getKBest(k, data, target):
    # selection
    selector = SelectKBest(score_func=mutual_info_classif, k=k)
    x_kbest = selector.fit_transform(data, target)
    # Create a DataFrame with the selected features
    kbest_features = data.columns[selector.get_support()]
    x_kbest_df = pd.DataFrame(x_kbest, columns=kbest_features)
    return x_kbest_df

def process_kbest(data, target, k, file_path):
    try:
        x_kbest_df = pd.read_csv(file_path)
    except FileNotFoundError:
        x_kbest_df = getKBest(k, data, target)
        x_kbest_df[target.name] = target.values
        x_kbest_df.to_csv(file_path, index=False)
    x_kbest_df = x_kbest_df.drop(columns=[target.name], axis=1)
    x_train, x_test, y_train, y_test = train_test_split(
        x_kbest_df, target, test_size=0.3, random_state=42, stratify=target
    )
    rf_model.fit(x_train, y_train)
    y_pred = rf_model.predict(x_test)
    accuracy, precision, recall, f1 = calculate_metrics(y_test, y_pred)
    kbest_features_names = x_kbest_df.columns.tolist()
    return (k, accuracy, precision, recall, f1, kbest_features_names)

def getRFE(k, data, target):
    # selection
    selector = RFE(estimator=rf_model, n_features_to_select=k)
    x_rfe = selector.fit_transform(data, target)
    # Create a DataFrame with the selected features
    rfe_features = data.columns[selector.get_support()]
    x_rfe_df = pd.DataFrame(x_rfe, columns=rfe_features)
    return x_rfe_df

def process_rfe(data, target, k, file_path):
    try:
        x_rfe_df = pd.read_csv(file_path)
    except FileNotFoundError:
        x_rfe_df = getRFE(k, data, target)
        x_rfe_df[target.name] = target.values
        x_rfe_df.to_csv(file_path, index=False)
    x_rfe_df = x_rfe_df.drop(columns=[target.name], axis=1)
    x_train, x_test, y_train, y_test = train_test_split(
        x_rfe_df, target, test_size=0.3, random_state=42, stratify=target
    )
    rf_model.fit(x_train, y_train)
    y_pred = rf_model.predict(x_test)
    accuracy, precision, recall, f1 = calculate_metrics(y_test, y_pred)
    rfe_features_names = x_rfe_df.columns.tolist()
    return (k, accuracy, precision, recall, f1, rfe_features_names)

def apply_boruta(data, target):
    x_train, x_test, y_train, y_test = train_test_split(
        data, target, test_size=0.3, random_state=42, stratify=target
    )
    boruta = BorutaPy(
        rf_model,
        n_estimators="auto",
        verbose=2,
        random_state=42,
    )
    boruta.fit(x_train.values, y_train.values)
    sel_x_train = boruta.transform(x_train.values)
    sel_x_test = boruta.transform(x_test.values)
    rf_model.fit(sel_x_train, y_train)
    y_pred = rf_model.predict(sel_x_test)
    accuracy, precision, recall, f1 = calculate_metrics(y_test, y_pred)
    selected_features = x_train.columns[boruta.support_].tolist()
    return (accuracy, precision, recall, f1, selected_features)

#### 4.3.1 Variance Threshold

In [ ]:
variances_df, features_high_variance, features_low_variance = getVariances(
    0, df_balanced_noTarget
)

# Plot variances of all features
plt.figure(figsize=(12, 6))
plt.bar(variances_df.index, variances_df['Variance'], color='skyblue')
plt.xticks(rotation=90)
plt.xlabel('Features')
plt.ylabel('Variance')
plt.title('Feature Variances')
plt.tight_layout()
plt.show()

# Highlight features with high and low variance
print("\nFeatures with High Variance:")
print(features_high_variance)
print("\nFeatures with Low Variance:")
print(features_low_variance)

#### 4.3.2 Correlation Matrix

In [ ]:
# Calculate correlation matrix
correlation_matrix = df_balanced.corr()

# Plot correlation heatmap
plt.figure(figsize=(14, 12))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Feature Correlation Heatmap")
plt.tight_layout()
plt.show()

# Focus on correlations with the target variable
diabetes_correlations = correlation_matrix['Diabetes_binary'].sort_values(ascending=False)
print("\nFeatures correlated with Diabetes (sorted):")
print(diabetes_correlations)

# Visualize correlations with the target
plt.figure(figsize=(12, 8))
diabetes_correlations[1:].plot(kind='bar')  # Exclude self-correlation
plt.title("Feature Correlation with Diabetes")
plt.xlabel("Features")
plt.ylabel("Correlation Coefficient")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

#### 4.3.3 KBest

In [ ]:
# results = []
# columns = ["Model", "K", "Accuracy", "Precision", "Recall", "F1-Score", "Features"]
# for k in range(1, df_balanced_noTarget.shape[1] + 1):
#     file_path = f"featureSelectionData/kbest/k/{k}best_features.csv"
#     kbestResult = process_kbest(df_balanced_noTarget, y_balanced, k, file_path)
#     results.append(("rf",) + kbestResult)
# save_results(
#     results, f"featureSelectionData/kbest/kbest_rf_results.csv", columns
# )

#### 4.3.4 RFE

In [ ]:
# results = []
# columns = ["Model", "K", "Accuracy", "Precision", "Recall", "F1-Score", "Features"]
# for k in range(1, df_balanced_noTarget.shape[1] + 1):
#     file_path = f"featureSelectionData/rfe/k/{k}rfe_features.csv"
#     rfeResult = process_rfe(df_balanced_noTarget, y_balanced, k, file_path)
#     results.append(("rf",) + rfeResult)
# save_results(results, f"featureSelectionData/rfe/rfe_rf_results.csv", columns)


#### 4.3.5 Boruta

In [ ]:
# columns = ["Model", "Accuracy", "Precision", "Recall", "F1-Score", "Features"]
# accuracy, precision, recall, f1, selected_features = apply_boruta(
#     df_balanced_noTarget, y_balanced
# )
# save_results(
#     [("rf", accuracy, precision, recall, f1, selected_features)],
#     f"featureSelectionData/boruta/boruta_rf_results.csv",
#     columns,
# )

### 4.4 Tuning model's parameters

In [ ]:
# from sklearn.model_selection import GridSearchCV

# rf_model = RandomForestClassifier(random_state=42, n_jobs=-1)

# params = {
#     'max_depth': [2,3,5,10,20],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [5,10,20,50,100,200],
#     'n_estimators': [50,100,150,200]
# }

# # Instantiate the grid search model
# grid_search = GridSearchCV(estimator=rf_model,
#                            param_grid=params,
#                            cv = 4,
#                            n_jobs=-1, verbose=1, scoring="precision")

# important_features = ['HighBP', 'HighChol', 'BMI', 'GenHlth', 'Age', 'Income']
# X_train_important = X_train_bal[important_features]

# grid_search.fit(X_train_important, y_train_bal)
# print("\nBest parameters found: ", grid_search.best_estimator_)
# rf_model = grid_search.best_estimator_

## 5 . Machine Learning Model without Differential Privacy

- Precision: How many predicted positives are actually positive?
- Recall: How many actual positives were correctly identified?
- F1-score: A balance between precision and recall.
- Support: Number of actual occurrences of each class.

Single-layer networks have just one layer of active units. Inputs connect directly to the outputs through a single layer of weights.

In multi-layer networks (MLP) there is a layer of input nodes, a layer of output nodes, and one or more intermediate (hidden) layers.

- Input Layer: Input variables, sometimes called the visible layer.
- Hidden Layers: Layers of nodes between the input and output layers. There may be one or more of these layers.
- Output Layer: A layer of nodes that produce the output variables.
- Size: The number of nodes in the model.
- Width: The number of nodes in a specific layer.
- Depth: The number of layers in a neural network.

The structure of an MLP can be summarized using a simple notation: the number of nodes in each layer is specified as an integer, in order from the input layer to the output layer, with the size of each layer separated by a "/". For example, a model with 3 input nodes, 2 hidden layers with 4 and 3 nodes respectively, and 1 output node would be represented as "3/4/3/1".

Steps to train a deep learning model with pytorch:
1. Scale the data
2. Label encode the target
3. Convert the data to tensors
4. Create a model
   1. Should we use a wide model or a deep model? 
   2. How many layers and neurons per layer should we use?
   3. How to choose? K-fold cross validaton
5. Train the model

K-fold cross validaton is a technique that, use a “training set” of data to train the model and then use a “test set” of data to see how accurate the model can predict. The result from test set is what we should focus on. But we do not want to test a model once because if we see an extremely good or bad result, it may be by chance. we want to run this process times with different training and test sets, such that we are ensured that we are comparing the model design, not the result of a particular training. K-fold cross validaton splits a larger dataset into portions and take one portion as the test set while the portions are combined as the training set. There are different such combinations. Therefore we can repeat the experiment for times and take the average result. To ensure that each portion contains equal number of classes we will use stratified k-fold cross validation.

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
import numpy as np

models_performance_path = os.path.join(results_models_dir, 'models_performance.csv')

# Define the model
class FeedForwardNN(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 32)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(32, 2)  # Binary classification

    def forward(self, x):
        x = self.relu(self.fc1(x))
        return self.fc2(x)

def get_train_test_tensors(X_train, y_train, X_test, y_test):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
    X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)
    
    return X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor

def getLoader(X_train_tensor, y_train_tensor):
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    return train_loader

def save_model_performance(classification_report_dict, model_name):
    performance = {
        "Model": model_name,
        "Accuracy": classification_report_dict["accuracy"],
        "Precision_0": classification_report_dict["0"]["precision"],
        "Precision_1": classification_report_dict["1"]["precision"],
        "Precision_weighted": classification_report_dict["weighted avg"]["precision"],
        "Recall_0": classification_report_dict["0"]["recall"],
        "Recall_1": classification_report_dict["1"]["recall"],
        "Recall_weighted": classification_report_dict["weighted avg"]["recall"],
        "F1_0": classification_report_dict["0"]["f1-score"],
        "F1_1": classification_report_dict["1"]["f1-score"],
        "F1_weighted": classification_report_dict["weighted avg"]["f1-score"],
        "Support_0": classification_report_dict["0"]["support"],
        "Support_1": classification_report_dict["1"]["support"],
        "Support_weighted": classification_report_dict["weighted avg"]["support"],
    }
    if os.path.isfile(models_performance_path,):
        existing_data = pd.read_csv(models_performance_path)
        if performance["Model"] not in existing_data["Model"].values:
            save_file(pd.DataFrame([performance]), models_performance_path, {'mode': 'a', 'header': False})
        else:
            existing_data.loc[existing_data["Model"] == performance["Model"], :] = pd.DataFrame([performance]).values
            save_file(existing_data, models_performance_path)
    else:
        save_file(pd.DataFrame([performance]), models_performance_path)



### 5.1 Using all features (balanced dataset)

In [ ]:
# Convert the data to tensors PyTorch tensors as this is the format a PyTorch model needs
train_test_tensors_allF_bal = get_train_test_tensors(X_train_bal, y_train_bal, 
                                                     X_test_bal, y_test_bal)
X_train_tensor_allF_bal = train_test_tensors_allF_bal[0]
y_train_tensor_allF_bal = train_test_tensors_allF_bal[1]
X_test_tensor_allF_bal = train_test_tensors_allF_bal[2]
y_test_tensor_allF_bal = train_test_tensors_allF_bal[3]

train_loader_allF_bal = getLoader(X_train_tensor_allF_bal, y_train_tensor_allF_bal)

model_allF_bal = FeedForwardNN(input_dim=X_train_tensor_allF_bal.shape[1])
optimizer_allF_bal = torch.optim.SGD(model_allF_bal.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Train the model
num_epochs = 5
print("Training neural network model...")
for epoch in range(num_epochs):
    model_allF_bal.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader_allF_bal:
        optimizer_allF_bal.zero_grad()
        output = model_allF_bal(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer_allF_bal.step()
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader_allF_bal):.4f}")

# Evaluate the model
model_allF_bal.eval()
with torch.no_grad():
    predictions = model_allF_bal(X_test_tensor_allF_bal)
    predicted_classes = torch.argmax(predictions, dim=1)
    
    # Convert to numpy for sklearn metrics
    y_pred = predicted_classes.numpy()
    y_true = y_test_tensor_allF_bal.numpy()
    
    # Calculate accuracy
    acc = accuracy_score(y_true, y_pred)
    print(f"Accuracy: {acc:.4f}")
    
    # Classification report
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred))

save_file(model_allF_bal, os.path.join(models_dir, 'deepL_allF_bal_model.pkl'))
# Save model performance
classification_report_dict = classification_report(y_true, y_pred, output_dict=True)
save_model_performance(classification_report_dict, 'deepL_allF_bal')

### 5.2 Using manually selected features (balanced dataset)

In [ ]:
# Define important features
manual_important_features = ['GenHlth', 'HighBP', 'DiffWalk', 'BMI', 'HighChol', 'Age', 
                      'HeartDiseaseorAttack', 'PhysHlth', 'Stroke', 'MentHlth', 
                      'CholCheck', 'Smoker', 'NoDocbcCost', 'Sex', 'AnyHealthcare', 
                      'Income', 'Education']

X_train_bal_manSelF = X_train_bal[manual_important_features]
X_test_bal_manSelF = X_test_bal[manual_important_features]
train_test_tensors_manSelF_bal = get_train_test_tensors(X_train_bal_manSelF, y_train_bal,
                                                     X_test_bal_manSelF, y_test_bal)
X_train_tensor_manSelF_bal = train_test_tensors_manSelF_bal[0]
y_train_tensor_manSelF_bal = train_test_tensors_manSelF_bal[1]
X_test_tensor_manSelF_bal = train_test_tensors_manSelF_bal[2]
y_test_tensor_manSelF_bal = train_test_tensors_manSelF_bal[3]

train_loader_manSelF_bal = getLoader(X_train_tensor_manSelF_bal, y_train_tensor_manSelF_bal)

model_manSelF_bal = FeedForwardNN(input_dim=X_train_tensor_manSelF_bal.shape[1])
optimizer_manSelF_bal = torch.optim.SGD(model_manSelF_bal.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Train the model
num_epochs = 5
print("Training neural network model with important features...")
for epoch in range(num_epochs):
    model_manSelF_bal.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader_manSelF_bal:
        optimizer_manSelF_bal.zero_grad()
        output = model_manSelF_bal(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer_manSelF_bal.step()
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader_manSelF_bal):.4f}")

# Evaluate the model
model_manSelF_bal.eval()
with torch.no_grad():
    predictions = model_manSelF_bal(X_test_tensor_manSelF_bal)
    predicted_classes = torch.argmax(predictions, dim=1)
    
    # Convert to numpy for sklearn metrics
    y_pred = predicted_classes.numpy()
    y_true = y_test_tensor_manSelF_bal.numpy()
    
    # Calculate accuracy
    acc = accuracy_score(y_true, y_pred)
    print(f"Accuracy: {acc:.4f}")
    
    # Classification report
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred))

save_file(model_manSelF_bal, os.path.join(models_dir, 'deepL_manSelF_bal_model.pkl'))
# Save model performance
classification_report_dict = classification_report(y_true, y_pred, output_dict=True)
save_model_performance(classification_report_dict, 'deepL_manSelF_bal')

### 5.3 Using selected features (balanced dataset)

In [ ]:
# Define the smaller set of important features
important_features = ['HighBP', 'HighChol', 'BMI', 'GenHlth', 'Age', 'Income']

X_train_bal_selF = X_train_bal[important_features]
X_test_bal_selF = X_test_bal[important_features]
train_test_tensors_selF_bal = get_train_test_tensors(X_train_bal_selF, y_train_bal,
                                                     X_test_bal_selF, y_test_bal)
X_train_tensor_selF_bal = train_test_tensors_selF_bal[0]
y_train_tensor_selF_bal = train_test_tensors_selF_bal[1]
X_test_tensor_selF_bal = train_test_tensors_selF_bal[2]
y_test_tensor_selF_bal = train_test_tensors_selF_bal[3]

train_loader_selF_bal = getLoader(X_train_tensor_selF_bal, y_train_tensor_selF_bal)

model_selF_bal = FeedForwardNN(input_dim=X_train_tensor_selF_bal.shape[1])
optimizer_selF_bal = torch.optim.SGD(model_selF_bal.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Train the model
num_epochs = 5
print("Training neural network model with selected features...")
for epoch in range(num_epochs):
    model_selF_bal.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader_selF_bal:
        optimizer_selF_bal.zero_grad()
        output = model_selF_bal(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer_selF_bal.step()
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader_selF_bal):.4f}")

# Evaluate the model
model_selF_bal.eval()
with torch.no_grad():
    predictions = model_selF_bal(X_test_tensor_selF_bal)
    predicted_classes = torch.argmax(predictions, dim=1)
    
    # Convert to numpy for sklearn metrics
    y_pred = predicted_classes.numpy()
    y_true = y_test_tensor_selF_bal.numpy()
    
    # Calculate accuracy
    acc = accuracy_score(y_true, y_pred)
    print(f"Accuracy: {acc:.4f}")
    
    # Classification report
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred))

save_file(model_selF_bal, os.path.join(models_dir, 'deepL_selF_bal_model.pkl'))
# Save model performance
classification_report_dict = classification_report(y_true, y_pred, output_dict=True)
save_model_performance(classification_report_dict, 'deepL_selF_bal')

### 5.4 Using selected features (imbalanced dataset)

**This model over prioritizes the majority class (0 - no diabetes), as it was trained with the imbalanced data.**

Comparison:

Balanced data (6.2) - Lower accuracy (74%) but much better at detecting class 1 - diabetes (79%), as it gives the same importance to both classes.

Imbalanced data (6.3) - Higher accuracy (86%) but poor detection of class 1 - diabetes (16%).

In [ ]:
# Filter the imbalanced training and test sets to include only important features
X_train_imb_selF = X_train_imb[important_features]
X_test_imb_selF = X_test_imb[important_features]
train_test_tensors_selF_imb = get_train_test_tensors(X_train_imb_selF, y_train_imb,
                                                     X_test_imb_selF, y_test_imb)
X_train_tensor_selF_imb = train_test_tensors_selF_imb[0]
y_train_tensor_selF_imb = train_test_tensors_selF_imb[1]
X_test_tensor_selF_imb = train_test_tensors_selF_imb[2]
y_test_tensor_selF_imb = train_test_tensors_selF_imb[3]

train_loader_selF_imb = getLoader(X_train_tensor_selF_imb, y_train_tensor_selF_imb)

model_selF_imb = FeedForwardNN(input_dim=X_train_tensor_selF_imb.shape[1])
optimizer_selF_imb = torch.optim.SGD(model_selF_imb.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Train the model
num_epochs = 5
print("Training neural network model with selected features on imbalanced data...")
for epoch in range(num_epochs):
    model_selF_imb.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader_selF_imb:
        optimizer_selF_imb.zero_grad()
        output = model_selF_imb(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer_selF_imb.step()
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader_selF_imb):.4f}")

# Evaluate the model
model_selF_imb.eval()
with torch.no_grad():
    predictions = model_selF_imb(X_test_tensor_selF_imb)
    predicted_classes = torch.argmax(predictions, dim=1)
    
    # Convert to numpy for sklearn metrics
    y_pred = predicted_classes.numpy()
    y_true = y_test_tensor_selF_imb.numpy()
    
    # Calculate accuracy
    acc = accuracy_score(y_true, y_pred)
    print(f"Accuracy: {acc:.4f}")
    
    # Check class distribution in predictions
    unique, counts = np.unique(y_pred, return_counts=True)
    pred_distribution = dict(zip(unique, counts))
    print(f"Prediction distribution: {pred_distribution}")
    
    # Classification report
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred))
save_file(model_selF_imb, os.path.join(models_dir, 'deepL_selF_imb_model.pkl'))
# Save model performance
classification_report_dict = classification_report(y_true, y_pred, output_dict=True)
save_model_performance(classification_report_dict, 'deepL_selF_imb')

#### 5.4.1 Using validation set

In [ ]:
# Filter datasets to include only important features
X_train_bal_selF = X_train_bal[important_features]
X_test_bal_selF = X_test_bal[important_features]
X_val_bal_selF = X_val_bal[important_features]

# Scale the data
scaler = StandardScaler()
X_train_scaled_selF = scaler.fit_transform(X_train_bal_selF)
X_test_scaled_selF = scaler.transform(X_test_bal_selF)
X_val_scaled_selF = scaler.transform(X_val_bal_selF)

# Convert to PyTorch tensors
X_train_tensor_selF = torch.tensor(X_train_scaled_selF, dtype=torch.float32)
y_train_tensor_selF = torch.tensor(y_train_bal.values, dtype=torch.long)

X_test_tensor_selF = torch.tensor(X_test_scaled_selF, dtype=torch.float32)
y_test_tensor_selF = torch.tensor(y_test_bal.values, dtype=torch.long)

X_val_tensor_selF = torch.tensor(X_val_scaled_selF, dtype=torch.float32)
y_val_tensor_selF = torch.tensor(y_val_bal.values, dtype=torch.long)

# Create dataset and dataloader
train_dataset_selF = TensorDataset(X_train_tensor_selF, y_train_tensor_selF)
train_loader_selF = DataLoader(train_dataset_selF, batch_size=64, shuffle=True)

# Initialize model with input dimension matching the number of selected features
model_selF = FeedForwardNN(input_dim=len(important_features))
criterion = nn.CrossEntropyLoss()
optimizer_selF = torch.optim.SGD(model_selF.parameters(), lr=0.01)

# Train the model
num_epochs = 5
print("Training neural network model with selected features and validation...")
for epoch in range(num_epochs):
    # Training
    model_selF.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader_selF:
        optimizer_selF.zero_grad()
        output = model_selF(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer_selF.step()
        running_loss += loss.item()
    
    # Validation
    model_selF.eval()
    with torch.no_grad():
        val_output = model_selF(X_val_tensor_selF)
        val_loss = criterion(val_output, y_val_tensor_selF)
        val_predicted = torch.argmax(val_output, dim=1)
        val_acc = (val_predicted == y_val_tensor_selF).float().mean().item()
    
    print(f"Epoch {epoch+1}, Training Loss: {running_loss/len(train_loader_selF):.4f}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")

# Final evaluation on test set
model_selF.eval()
with torch.no_grad():
    predictions = model_selF(X_test_tensor_selF)
    predicted_classes = torch.argmax(predictions, dim=1)
    
    # Convert to numpy for sklearn metrics
    y_pred = predicted_classes.numpy()
    y_true = y_test_tensor_selF.numpy()
    
    # Calculate accuracy
    test_acc = accuracy_score(y_true, y_pred)
    print(f"\nBalanced Test Accuracy: {test_acc:.4f}")
    
    # Classification report
    print("\nBalanced Test Classification Report:")
    print(classification_report(y_true, y_pred))

# Evaluate validation set
with torch.no_grad():
    val_predictions = model_selF(X_val_tensor_selF)
    val_predicted_classes = torch.argmax(val_predictions, dim=1)
    
    # Convert to numpy for sklearn metrics
    y_pred_val = val_predicted_classes.numpy()
    y_val_true = y_val_tensor_selF.numpy()
    
    # Calculate accuracy
    val_acc = accuracy_score(y_val_true, y_pred_val)
    print(f"\nBalanced Validation Accuracy: {val_acc:.4f}")
    
    # Classification report
    print("\nBalanced Validation Classification Report:")
    print(classification_report(y_val_true, y_pred_val))

#### 5.4.2 Using SMOTE balancing

In [ ]:
#TODO: put smote balancing in preprocessing?

from imblearn.over_sampling import SMOTE
from collections import Counter

X = df[important_features]
y = df['Diabetes_binary']

# Handle class imbalance using SMOTE
# SMOTE balances the dataset by generating synthetic samples for the minority class
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

print("Before SMOTE:", Counter(y))  # Shows original class distribution
print("After SMOTE:", Counter(y_resampled))  # Shows balanced class distribution

print("Original dataset shape:", X.shape, y.shape)
print("Resampled dataset shape:", X_resampled.shape, y_resampled.shape)

# Train-test split
X_train, X_temp, y_train, y_temp = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42, stratify=y_resampled)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=2/3, random_state=42, stratify=y_temp)

# Train with the balanced dataset split
rf_model.fit(X_train, y_train)

# Make predictions
y_pred = rf_model.predict(X_test)

# Checking accuracy and classification report
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

### 5.5 Applying Differential Privacy Techniques

#### 5.5.1 Gradient Descent

In [ ]:
import opacus

print(np.__version__) # min 1.21
print(opacus.__version__) # min 1.3

##### 5.5.1.1 allF_bal

In [ ]:
train_test_tensors_allF_bal = get_train_test_tensors(X_train_bal, y_train_bal, 
                                                     X_test_bal, y_test_bal)
X_train_tensor_allF_bal = train_test_tensors_allF_bal[0]
y_train_tensor_allF_bal = train_test_tensors_allF_bal[1]
X_test_tensor_allF_bal = train_test_tensors_allF_bal[2]
y_test_tensor_allF_bal = train_test_tensors_allF_bal[3]

train_loader_allF_bal = getLoader(X_train_tensor_allF_bal, y_train_tensor_allF_bal)

model_allF_bal = FeedForwardNN(input_dim=X_train_tensor_allF_bal.shape[1])
optimizer_allF_bal = torch.optim.SGD(model_allF_bal.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

privacy_engine = opacus.PrivacyEngine()
model_allF_bal, optimizer_allF_bal, train_loader_allF_bal = privacy_engine.make_private_with_epsilon(
    module=model_allF_bal,
    optimizer=optimizer_allF_bal,
    data_loader=train_loader_allF_bal,
    target_epsilon=5.0,     # Set desired privacy budget
    target_delta=1e-5,
    max_grad_norm=1.0,
    epochs=5
)

# Train the model
model_allF_bal.train()
for epoch in range(5):
    for X_batch, y_batch in train_loader_allF_bal:
        optimizer_allF_bal.zero_grad()
        output = model_allF_bal(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer_allF_bal.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")
    
# Evaluate the model
model_allF_bal.eval()
with torch.no_grad():
    predictions = model_allF_bal(X_test_tensor_allF_bal)
    predicted_classes = torch.argmax(predictions, dim=1)
    # Convert to numpy for sklearn metrics
    y_pred = predicted_classes.numpy()
    y_true = y_test_tensor_allF_bal.numpy()
    # Calculate accuracy
    acc_allF_bal = accuracy_score(y_true, y_pred)
    print(f"Accuracy (all features): {acc_allF_bal:.4f}")
    # Classification report
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred))
    

epsilon = privacy_engine.get_epsilon(delta=1e-5)
print(f"Final ε: {epsilon:.2f}")

save_file(model_allF_bal, os.path.join(models_dir, 'deepL_allF_bal_dp_model.pkl'))
# Save model performance
classification_report_dict = classification_report(y_true, y_pred, output_dict=True)
save_model_performance(classification_report_dict, 'deepL_allF_bal_dp')

##### 5.5.1.2 selF_bal

In [ ]:
X_train_bal_selF = X_train_bal[important_features]
X_test_bal_selF = X_test_bal[important_features]
train_test_tensors_selF_bal = get_train_test_tensors(X_train_bal_selF, y_train_bal,
                                                     X_test_bal_selF, y_test_bal)
X_train_tensor_selF_bal = train_test_tensors_selF_bal[0]
y_train_tensor_selF_bal = train_test_tensors_selF_bal[1]
X_test_tensor_selF_bal = train_test_tensors_selF_bal[2]
y_test_tensor_selF_bal = train_test_tensors_selF_bal[3]

train_loader_selF_bal = getLoader(X_train_tensor_selF_bal, y_train_tensor_selF_bal)

model_selF_bal = FeedForwardNN(input_dim=X_train_tensor_selF_bal.shape[1])
optimizer_selF_bal = torch.optim.SGD(model_selF_bal.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

privacy_engine = opacus.PrivacyEngine()
model_selF_bal, optimizer_selF_bal, train_loader_selF_bal = privacy_engine.make_private_with_epsilon(
    module=model_selF_bal,
    optimizer=optimizer_selF_bal,
    data_loader=train_loader_selF_bal,
    target_epsilon=5.0,     # Set desired privacy budget
    target_delta=1e-5,
    max_grad_norm=1.0,
    epochs=5
)

# Train the model
model_selF_bal.train()
for epoch in range(5):
    for X_batch, y_batch in train_loader_selF_bal:
        optimizer_selF_bal.zero_grad()
        output = model_selF_bal(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer_selF_bal.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")
    
# Evaluate the model
model_selF_bal.eval()
with torch.no_grad():
    predictions = model_selF_bal(X_test_tensor_selF_bal)
    predicted_classes = torch.argmax(predictions, dim=1)
    # Convert to numpy for sklearn metrics
    y_pred = predicted_classes.numpy()
    y_true = y_test_tensor_selF_bal.numpy()
    # Calculate accuracy
    acc_selF_bal = accuracy_score(y_true, y_pred)
    print(f"Accuracy (selected features): {acc_selF_bal:.4f}")
    # Classification report
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred))

epsilon = privacy_engine.get_epsilon(delta=1e-5)
print(f"Final ε: {epsilon:.2f}")

save_file(model_selF_bal, os.path.join(models_dir, 'deepL_selF_bal_dp_model.pkl'))
# Save model performance
classification_report_dict = classification_report(y_true, y_pred, output_dict=True)
save_model_performance(classification_report_dict, 'deepL_selF_bal_dp')

##### 5.5.1.3 manSelF_bal

In [ ]:
X_train_bal_manSelF = X_train_bal[manual_important_features]
X_test_bal_manSelF = X_test_bal[manual_important_features]
train_test_tensors_manSelF_bal = get_train_test_tensors(X_train_bal_manSelF, y_train_bal, 
                                                        X_test_bal_manSelF, y_test_bal)
X_train_tensor_manSelF_bal = train_test_tensors_manSelF_bal[0]
y_train_tensor_manSelF_bal = train_test_tensors_manSelF_bal[1]
X_test_tensor_manSelF_bal = train_test_tensors_manSelF_bal[2]
y_test_tensor_manSelF_bal = train_test_tensors_manSelF_bal[3]

train_loader_manSelF_bal = getLoader(X_train_tensor_manSelF_bal, y_train_tensor_manSelF_bal)

model_manSelF_bal = FeedForwardNN(input_dim=X_train_tensor_manSelF_bal.shape[1])
optimizer_manSelF_bal = torch.optim.SGD(model_manSelF_bal.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

privacy_engine = opacus.PrivacyEngine()
model_manSelF_bal, optimizer_manSelF_bal, train_loader_manSelF_bal = privacy_engine.make_private_with_epsilon(
    module=model_manSelF_bal,
    optimizer=optimizer_manSelF_bal,
    data_loader=train_loader_manSelF_bal,
    target_epsilon=5.0,     # Set desired privacy budget
    target_delta=1e-5,
    max_grad_norm=1.0,
    epochs=5
)

# Train the model
model_manSelF_bal.train()
for epoch in range(5):
    for X_batch, y_batch in train_loader_manSelF_bal:
        optimizer_manSelF_bal.zero_grad()
        output = model_manSelF_bal(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer_manSelF_bal.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")
    
# Evaluate the model
model_manSelF_bal.eval()
with torch.no_grad():
    predictions = model_manSelF_bal(X_test_tensor_manSelF_bal)
    predicted_classes = torch.argmax(predictions, dim=1)
    # Convert to numpy for sklearn metrics
    y_pred = predicted_classes.numpy()
    y_true = y_test_tensor_manSelF_bal.numpy()
    # Calculate accuracy
    acc_manSelF_bal = accuracy_score(y_true, y_pred)
    print(f"Accuracy (selected features): {acc_manSelF_bal:.4f}")
    # Classification report
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred))

epsilon = privacy_engine.get_epsilon(delta=1e-5)
print(f"Final ε: {epsilon:.2f}")

save_file(model_manSelF_bal, os.path.join(models_dir, 'deepL_manSelF_bal_dp_model.pkl'))
# Save model performance
classification_report_dict = classification_report(y_true, y_pred, output_dict=True)
save_model_performance(classification_report_dict, 'deepL_manSelF_bal_dp')

## 6 Attacks

In [ ]:
import copy

# File paths for all models
model_files = [
    'models/deepL_allF_bal_model.pkl',
    'models/deepL_selF_bal_model.pkl',
    'models/deepL_manSelF_bal_model.pkl',
    'models/deepL_allF_bal_dp_model.pkl',
    'models/deepL_selF_bal_dp_model.pkl',
    'models/deepL_manSelF_bal_dp_model.pkl',
]

# Load all models
with open(model_files[0], 'rb') as file:
    deepL_model_allF = pickle.load(file)
with open(model_files[1], 'rb') as file:
    deepL_model_selF = pickle.load(file)
with open(model_files[2], 'rb') as file:
    deepL_model_manSelF = pickle.load(file)
with open(model_files[3], 'rb') as file:
    deepL_model_allF_dp = pickle.load(file)
with open(model_files[4], 'rb') as file:
    deepL_model_selF_dp = pickle.load(file)
with open(model_files[5], 'rb') as file:
    deepL_model_manSelF_dp = pickle.load(file)

#### 6.1 Model Inversion Attack

In [ ]:
# Get models
deepL_model_allF_copy = copy.deepcopy(deepL_model_allF)
deepL_model_manSelF_copy = copy.deepcopy(deepL_model_manSelF)
deepL_model_selF_copy = copy.deepcopy(deepL_model_selF)
deepL_model_allF_dp_copy = copy.deepcopy(deepL_model_allF_dp)
deepL_model_selF_dp_copy = copy.deepcopy(deepL_model_selF_dp)
deepL_model_manSelF_dp_copy = copy.deepcopy(deepL_model_manSelF_dp)

In [ ]:
def model_inversion_attack_nn(model, target_class, feature_names, X_train, scaler=None, iterations=300):
    # Set model to evaluation mode
    model.eval()
    
    # Get min/max values for each feature
    min_values = X_train[feature_names].min()
    max_values = X_train[feature_names].max()
    
    # Start with random values for the features
    sample = {}
    for feature in feature_names:
        sample[feature] = np.random.uniform(min_values[feature], max_values[feature])
    
    # Save best sample and confidence
    best_sample = sample.copy()
    best_confidence = 0
    
    # Optimize sample through iterations
    step_size = 0.05
    for i in range(iterations):
        # Create DataFrame and convert to tensor
        current_df = pd.DataFrame([sample])
        
        # Scale the data if a scaler is provided
        if scaler:
            current_data = scaler.transform(current_df)
        else:
            current_data = current_df.values
        
        # Convert to PyTorch tensor
        current_tensor = torch.tensor(current_data, dtype=torch.float32)
        
        # Get prediction
        with torch.no_grad():
            output = model(current_tensor)
            probabilities = torch.nn.functional.softmax(output, dim=1)
            confidence = probabilities[0][target_class].item()
        
        # Save if better than previous
        if confidence > best_confidence:
            best_confidence = confidence
            best_sample = sample.copy()
        
        # Optimize one feature at a time
        for feature in feature_names:
            original = sample[feature]
            
            # Try positive change
            sample[feature] = min(max_values[feature], original + step_size)
            
            # Create and scale data
            plus_df = pd.DataFrame([sample])
            if scaler:
                plus_data = scaler.transform(plus_df)
            else:
                plus_data = plus_df.values
                
            # Get prediction
            plus_tensor = torch.tensor(plus_data, dtype=torch.float32)
            with torch.no_grad():
                plus_output = model(plus_tensor)
                plus_probs = torch.nn.functional.softmax(plus_output, dim=1)
                plus_conf = plus_probs[0][target_class].item()
            
            # Try negative change
            sample[feature] = max(min_values[feature], original - step_size)
            
            # Create and scale data
            minus_df = pd.DataFrame([sample])
            if scaler:
                minus_data = scaler.transform(minus_df)
            else:
                minus_data = minus_df.values
                
            # Get prediction
            minus_tensor = torch.tensor(minus_data, dtype=torch.float32)
            with torch.no_grad():
                minus_output = model(minus_tensor)
                minus_probs = torch.nn.functional.softmax(minus_output, dim=1)
                minus_conf = minus_probs[0][target_class].item()
            
            # Choose the best direction
            if plus_conf > minus_conf and plus_conf > confidence:
                sample[feature] = min(max_values[feature], original + step_size)
            elif minus_conf > plus_conf and minus_conf > confidence:
                sample[feature] = max(min_values[feature], original - step_size)
            else:
                sample[feature] = original
        
        # Reduce step size over time
        if i % 100 == 0 and i > 0:
            step_size *= 0.9
    
    # Find most similar training example
    reconstructed_df = pd.DataFrame([best_sample])
    
    # Calculate distance to each training example
    distances = []
    for _, row in X_train.iterrows():
        dist = 0
        for feature in feature_names:
            dist += (row[feature] - best_sample[feature])**2
        distances.append(np.sqrt(dist))
    
    most_similar_idx = np.argmin(distances)
    most_similar = X_train.iloc[[most_similar_idx]]
    
    return {
        'reconstructed': reconstructed_df,
        'most_similar': most_similar,
        'confidence': best_confidence,
        'similarity': 1.0 / (1.0 + np.min(distances))  # Convert distance to similarity (0-1)
    }

def analyze_model_inversion_attack(attack_results, feature_names):
    print(f"Attack completed with confidence: {attack_results['confidence']:.4f}")
    print(f"Similarity to closest training sample: {attack_results['similarity']:.4f}")
    
    # Visualize comparison between reconstructed and original sample
    plt.figure(figsize=(12, 8))
    
    # Get data for visualization
    recon = attack_results['reconstructed']
    orig = attack_results['most_similar']
    
    # Plot all features
    display_features = feature_names
    
    x = np.arange(len(display_features))
    width = 0.3
    
    plt.bar(x - width/2, [recon[f].values[0] for f in display_features], width, label='Reconstructed')
    plt.bar(x + width/2, [orig[f].values[0] for f in display_features], width, label='Original')
    
    plt.xlabel('Features')
    plt.ylabel('Values')
    plt.title('Model Inversion Attack: Reconstructed vs. Original Sample')
    plt.xticks(x, display_features, rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    # Compare all features in detail
    print("\n FEATURE-SPECIFIC ANALYSIS")
    for feature in feature_names:
        recon_value = recon[feature].values[0]
        orig_value = orig[feature].values[0]
        
        diff = abs(recon_value - orig_value)
        
        print(f"Feature: {feature}")
        print(f"  Original value: {orig_value:.2f}")
        print(f"  Reconstructed value: {recon_value:.2f}")
        print(f"  Absolute difference: {diff:.2f}")
        print()
    
    return attack_results

def run_attack_on_model(model, target_class, feature_names, X_train, scaler=None):
    # Run model inversion attack on a specific model
    results = model_inversion_attack_nn(
        model=model,
        target_class=target_class,
        feature_names=feature_names,
        X_train=X_train,
        scaler=scaler
    )
    
    analyze_model_inversion_attack(results, feature_names)
    return results

In [ ]:
def create_scaler_for_model(X_data):
    # Create and fit a StandardScaler for the data
    scaler = StandardScaler()
    scaler.fit(X_data)
    return scaler

# Create scalers for each model
scaler_all = create_scaler_for_model(X_train_bal)
scaler_manSelF = create_scaler_for_model(X_train_bal[manual_important_features])
scaler_selF = create_scaler_for_model(X_train_bal[important_features])

# Run attack on All Features Model
print("\nMODEL INVERSION ATTACK: ALL FEATURES")
results_all = model_inversion_attack_nn(
    model=deepL_model_allF_copy, 
    target_class=1, 
    feature_names=X_train_bal.columns.tolist(), 
    X_train=X_train_bal,
    scaler=scaler_all
)
analyze_model_inversion_attack(results_all, X_train_bal.columns.tolist())

# Run attack on Manual Selected Features Model
print("\nMODEL INVERSION ATTACK: MANUAL SELECTED FEATURES")
results_manSelF = model_inversion_attack_nn(
    model=deepL_model_manSelF_copy, 
    target_class=1, 
    feature_names=manual_important_features, 
    X_train=X_train_bal[manual_important_features],
    scaler=scaler_manSelF
)
analyze_model_inversion_attack(results_manSelF, manual_important_features)

# Run attack on Selected Features Model
print("\nMODEL INVERSION ATTACK: SELECTED FEATURES")
results_selF = model_inversion_attack_nn(
    model=deepL_model_selF_copy, 
    target_class=1, 
    feature_names=important_features, 
    X_train=X_train_bal[important_features],
    scaler=scaler_selF
)
analyze_model_inversion_attack(results_selF, important_features)

# Compare all models
print("\nATTACK EFFECTIVENESS COMPARISON")
print(f"{'Model':<25} {'Confidence':<15} {'Similarity':<15}")
print(f"{'-'*25:<25} {'-'*15:<15} {'-'*15:<15}")
print(f"{'All Features':<25} {results_all['confidence']:<15.4f} {results_all['similarity']:<15.4f}")
print(f"{'Manual Selected Features':<25} {results_manSelF['confidence']:<15.4f} {results_manSelF['similarity']:<15.4f}")
print(f"{'Selected Features':<25} {results_selF['confidence']:<15.4f} {results_selF['similarity']:<15.4f}")

#### 6.2 Membership Inference Attack(MIA) 

---1. Standard MIA---

The goal of this attack is to determine whether a given sample was used in training. 

It queries the model with different data points and analyzes the confidence scores. This happens because the model behaves differently for data it has seen before and new data. As this one was trained without DP, it is easier to leak private data as it may have "memorized" training data. A classification model would classify a training data record to its true class with a high confidence score while classifying a test data record to its true class with a relatively small confidence. These different behaviors of ML models enable an attacker of MIAs to build attack models to distinguish members from non-members of the training dataset.

The more overfitted a machine learning model is, the easier it will be for an adversary to stage membership inference attacks against it. Therefore, a machine model that generalizes well on unseen examples is also more secure against membership inference.

Membership inference attacks are not successful on all kinds of machine learning tasks. To create an efficient attack model, the adversary must be able to explore the feature space. (HigRes. photos: Hard, tabular data: Easy)

** TODO: try with DP model(s) as well **

In [ ]:
# Get models
deepL_model_allF_copy = copy.deepcopy(deepL_model_allF)
deepL_model_manSelF_copy = copy.deepcopy(deepL_model_manSelF)
deepL_model_selF_copy = copy.deepcopy(deepL_model_selF)
deepL_model_allF_dp_copy = copy.deepcopy(deepL_model_allF_dp)
deepL_model_selF_dp_copy = copy.deepcopy(deepL_model_selF_dp)
deepL_model_manSelF_dp_copy = copy.deepcopy(deepL_model_manSelF_dp)

In [ ]:
def dynamic_threshold(X_train, model, feature_names, scaler=None):
    """
    Calculate a dynamic threshold based on the maximum observed confidence 
    for samples in the training set.
    """
    max_confidence = 0
    for i in range(len(X_train)):
        sample = X_train.iloc[i][feature_names]
        df_sample = pd.DataFrame([sample])
        sample_data = scaler.transform(df_sample) if scaler else df_sample.values
        sample_tensor = torch.tensor(sample_data, dtype=torch.float32)

        # Get model prediction and confidence
        with torch.no_grad():
            output = model(sample_tensor)
            probabilities = torch.nn.functional.softmax(output, dim=1)
            confidence = probabilities[0].max().item()  # max confidence across all classes

        if confidence > max_confidence:
            max_confidence = confidence

    # Define a dynamic threshold based on a fraction of the maximum confidence
    return max_confidence * 0.8  # For example, 80% of the maximum confidence observed

def membership_inference_attack_nn(model, target_class, feature_names, X_train, scaler=None, iterations=300):
    model.eval()

    # Get min/max values for each feature
    min_values = X_train[feature_names].min()
    max_values = X_train[feature_names].max()

    # Initialize random sample
    sample = {feature: np.random.uniform(min_values[feature], max_values[feature]) for feature in feature_names}

    best_sample = sample.copy()
    best_confidence = 0
    confidence_history = []  # To track confidence over iterations
    step_size = 0.05

    # Calculate dynamic threshold
    dynamic_thresh = dynamic_threshold(X_train, model, feature_names, scaler)
    print(f"Dynamic Threshold: {dynamic_thresh:.4f}")

    # Optimize sample over iterations
    for i in range(iterations):
        current_df = pd.DataFrame([sample])
        current_data = scaler.transform(current_df) if scaler else current_df.values
        current_tensor = torch.tensor(current_data, dtype=torch.float32)

        # Get model prediction and confidence
        with torch.no_grad():
            output = model(current_tensor)
            probabilities = torch.nn.functional.softmax(output, dim=1)
            confidence = probabilities[0][target_class].item()

        confidence_history.append(confidence)

        # Save best sample and confidence
        if confidence > best_confidence:
            best_confidence = confidence
            best_sample = sample.copy()

        # Optimize sample by adjusting one feature at a time
        for feature in feature_names:
            original = sample[feature]

            # Try positive change
            sample[feature] = min(max_values[feature], original + step_size)
            plus_df = pd.DataFrame([sample])
            plus_data = scaler.transform(plus_df) if scaler else plus_df.values
            plus_tensor = torch.tensor(plus_data, dtype=torch.float32)
            with torch.no_grad():
                plus_output = model(plus_tensor)
                plus_conf = torch.nn.functional.softmax(plus_output, dim=1)[0][target_class].item()

            # Try negative change
            sample[feature] = max(min_values[feature], original - step_size)
            minus_df = pd.DataFrame([sample])
            minus_data = scaler.transform(minus_df) if scaler else minus_df.values
            minus_tensor = torch.tensor(minus_data, dtype=torch.float32)
            with torch.no_grad():
                minus_output = model(minus_tensor)
                minus_conf = torch.nn.functional.softmax(minus_output, dim=1)[0][target_class].item()

            # Choose the best direction
            if plus_conf > minus_conf and plus_conf > confidence:
                sample[feature] = min(max_values[feature], original + step_size)
            elif minus_conf > plus_conf and minus_conf > confidence:
                sample[feature] = max(min_values[feature], original - step_size)
            else:
                sample[feature] = original

        # Reduce step size over time for finer tuning
        if i % 100 == 0 and i > 0:
            step_size *= 0.9

    # Determine membership using dynamic threshold
    is_member = best_confidence > dynamic_thresh

    # Find the most similar training example
    reconstructed_df = pd.DataFrame([best_sample])

    # Fix here: Convert best_sample to NumPy array for comparison
    best_sample_values = np.array([best_sample[feature] for feature in feature_names])  # Convert to NumPy array
    distances = [np.linalg.norm(X_train[feature_names].iloc[idx].values - best_sample_values) for idx in range(X_train.shape[0])]

    most_similar_idx = np.argmin(distances)
    most_similar = X_train.iloc[[most_similar_idx]]

    return {
        'reconstructed': reconstructed_df,
        'most_similar': most_similar,
        'confidence': best_confidence,
        'membership': is_member,
        'similarity': 1.0 / (1.0 + np.min(distances)),
        'confidence_history': confidence_history  # Track confidence evolution
    }

def analyze_membership_inference_attack(attack_results, feature_names):
    print(f"Attack completed with confidence: {attack_results['confidence']:.4f}")
    print(f"Membership prediction: {'Member' if attack_results['membership'] else 'Not Member'}")
    print(f"Similarity to closest training sample: {attack_results['similarity']:.4f}")
    
    # Plot evolution of confidence over iterations
    plt.figure(figsize=(10, 6))
    plt.plot(range(len(attack_results['confidence_history'])), attack_results['confidence_history'], label='Confidence Evolution')
    plt.xlabel('Iteration')
    plt.ylabel('Confidence')
    plt.title('Confidence Evolution Over Attack Iterations')
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

def run_attack_on_model(model, target_class, feature_names, X_train, scaler=None):
    # Run membership inference attack on a specific model
    results = membership_inference_attack_nn(
        model=model,
        target_class=target_class,
        feature_names=feature_names,
        X_train=X_train,
        scaler=scaler
    )
    
    analyze_membership_inference_attack(results, feature_names)
    return results

def create_scaler_for_model(X_data):
    # Create and fit a StandardScaler for the data
    scaler = StandardScaler()
    scaler.fit(X_data)
    return scaler

# Example usage
scaler_all = create_scaler_for_model(X_train_bal)
scaler_manSelF = create_scaler_for_model(X_train_bal[manual_important_features])
scaler_selF = create_scaler_for_model(X_train_bal[important_features])

# Run attack on All Features Model
print("\nMEMBERSHIP INFERENCE ATTACK")
print("allF:")
results_all = run_attack_on_model(
    model=deepL_model_allF_copy, 
    target_class=1, 
    feature_names=X_train_bal.columns.tolist(), 
    X_train=X_train_bal,
    scaler=scaler_all
)
print("allF_dp:")
results_all_dp = run_attack_on_model(
    model=deepL_model_allF_dp_copy, 
    target_class=1, 
    feature_names=X_train_bal.columns.tolist(), 
    X_train=X_train_bal,
    scaler=scaler_all
)

# Run attack on Manual Selected Features Model
print("\nMEMBERSHIP INFERENCE ATTACK")
print("manSelF:")
results_manSelF = run_attack_on_model(
    model=deepL_model_manSelF_copy, 
    target_class=1, 
    feature_names=manual_important_features, 
    X_train=X_train_bal[manual_important_features],
    scaler=scaler_manSelF
)
print("manSelF_dp:")
results_manSelF_dp = run_attack_on_model(
    model=deepL_model_manSelF_dp_copy,
    target_class=1,
    feature_names=manual_important_features,
    X_train=X_train_bal[manual_important_features],
    scaler=scaler_manSelF
)

# Run attack on Selected Features Model
print("\nMEMBERSHIP INFERENCE ATTACK")
print("selF:")
results_selF = run_attack_on_model(
    model=deepL_model_selF_copy, 
    target_class=1, 
    feature_names=important_features, 
    X_train=X_train_bal[important_features],
    scaler=scaler_selF
)
print("selF_dp:")
results_selF_dp = run_attack_on_model(
    model=deepL_model_selF_dp_copy, 
    target_class=1, 
    feature_names=important_features, 
    X_train=X_train_bal[important_features],
    scaler=scaler_selF
)

# Compare all models
print("\nATTACK EFFECTIVENESS COMPARISON")
print(f"{'Model':<25} {'Confidence':<15} {'Similarity':<15}")
print(f"{'-'*25:<25} {'-'*15:<15} {'-'*15:<15}")
print("\nWithout DP:")
print(f"{'All Features':<25} {results_all['confidence']:<15.4f} {results_all['similarity']:<15.4f}")
print(f"{'Manual Selected Features':<25} {results_manSelF['confidence']:<15.4f} {results_manSelF['similarity']:<15.4f}")
print(f"{'Selected Features':<25} {results_selF['confidence']:<15.4f} {results_selF['similarity']:<15.4f}")
print("\nWith DP:")
print(f"{'All Features':<25} {results_all_dp['confidence']:<15.4f} {results_all_dp['similarity']:<15.4f}")
print(f"{'Manual Selected Features':<25} {results_manSelF_dp['confidence']:<15.4f} {results_manSelF_dp['similarity']:<15.4f}")
print(f"{'Selected Features':<25} {results_selF_dp['confidence']:<15.4f} {results_selF_dp['similarity']:<15.4f}")


As features are reduced, attack confidence drops, but similarity increases. This means that fewer, well-chosen features let the attack reconstruct samples closer to reality, even if less sure. The selected features expose more structure for the attacker.

TODO: maybe add the data of each reconstructed member in the output

--- 2. Real World Case MIA ---

In [ ]:
# # To keep: For development
# real_members = X_train.head(5)
# print("First 5 Real Members (Training Set):\n", real_members)

risky_patients = [
    # If adding more patients, make sure that the attributes have valid values according to the dataset

    {
        'GenHlth': 3, 'HighBP': 1, 'DiffWalk': 0, 'BMI': 21, 'HighChol': 0, 'Age': 12, 
        'HeartDiseaseorAttack': 0, 'PhysHlth': 0, 'Stroke': 0, 'MentHlth': 0, 'CholCheck': 1, 
        'Smoker': 0, 'Veggies': 1, 'HvyAlcoholConsump': 0, 'PhysActivity': 1, 'Education': 4, 
        'Income': 4
    },  # Patient used in the test_set. ID nr 178592

    {
        'GenHlth': 2, 'HighBP': 1, 'DiffWalk': 0, 'BMI': 39, 'HighChol': 1, 'Age': 7, 
        'HeartDiseaseorAttack': 0, 'PhysHlth': 0, 'Stroke': 0, 'MentHlth': 0, 'CholCheck': 1, 
        'Smoker': 0, 'Veggies': 1, 'HvyAlcoholConsump': 0, 'PhysActivity': 1, 'Education': 4, 
        'Income': 1
    },  # Patient used in the test_set. ID nr 318886

    {
        'GenHlth': 4, 'HighBP': 0, 'DiffWalk': 1, 'BMI': 23, 'HighChol': 0, 'Age': 8, 
        'HeartDiseaseorAttack': 1, 'PhysHlth': 24, 'Stroke': 0, 'MentHlth': 22, 'CholCheck': 0, 
        'Smoker': 1, 'Veggies': 0, 'HvyAlcoholConsump': 1, 'PhysActivity': 0, 'Education': 2, 
        'Income': 1
    },  # Random person created for testing purposes

    {
        'GenHlth': 5, 'HighBP': 1, 'DiffWalk': 1, 'BMI': 40, 'HighChol': 1, 'Age': 9, 
        'HeartDiseaseorAttack': 0, 'PhysHlth': 15, 'Stroke': 0, 'MentHlth': 18, 'CholCheck': 1, 
        'Smoker': 1, 'Veggies': 1, 'HvyAlcoholConsump': 0, 'PhysActivity': 0, 'Education': 4, 
        'Income': 3
    }   # Patient ID 0. Taken from the dataset. Probably not used in test_set? - ToDo
]

# Convert risky patients to pandas dataFrame as it is the expected input format by the deepL model
risky_df = pd.DataFrame(risky_patients)

risky_df = risky_df[X_train.columns]

# Predict confidence scores for risky patients
risky_confidences = rf_model.predict_proba(risky_df).max(axis=1)

# Determine membership status using the same attack threshold from the standard MIAttack
risky_membership = ['Member' if c > threshold else 'Non-Member' for c in risky_confidences]

for i, (conf, membership) in enumerate(zip(risky_confidences, risky_membership)):
    print(f"Risky Patient {i + 1}: Confidence = {conf:.2f}, Predicted Membership = {membership}")

plt.figure(figsize=(8, 5))
plt.hist(train_confidences, bins=50, alpha=0.6, color='blue', label="Train (Member)")
plt.hist(test_confidences, bins=50, alpha=0.6, color='red', label="Test (Non-Member)")

for i, conf in enumerate(risky_confidences):
    plt.axvline(conf, color='green', linestyle='dashed', label=f"Risky Patient {i + 1}: {conf:.2f}")

plt.axvline(threshold, color='black', linestyle='solid', label=f"Attack Threshold: {threshold:.2f}")
plt.xlabel("Model Confidence Score")
plt.ylabel("Count")
plt.title("Membership Inference Attack - Risky Patients")
plt.legend()
plt.show()


Add description, explain results and describe and how it can relate to a real world case (like an insurance company)

<!-- #### 6.3 Attribute Inference Attack -->

In [ ]:
# # Get models
# deepL_model_allF_copy = copy.deepcopy(deepL_model_allF)
# deepL_model_manSelF_copy = copy.deepcopy(deepL_model_manSelF)
# deepL_model_selF_copy = copy.deepcopy(deepL_model_selF)

In [ ]:
# from tabulate import tabulate

# assert 'rf_model' in locals(), "Deep Learning model (rf_model) is not defined!" #if not defined, run 6.6

# sensitive_feature = 'Income'

# # Exclude 'Income' from training features - because we will be trying to infer this attribute
# attack_features = [f for f in X_train.columns if f != sensitive_feature]

# # Get confidence scores from the target model
# # For each prediction, predict_proba() gives an array of probabilities for each class (8 in case of Income)
# train_confidences = rf_model.predict_proba(X_train).max(axis=1)
# test_confidences = rf_model.predict_proba(X_test).max(axis=1)

# # Add confidence scores to the dataset
# X_train_attack = X_train[attack_features].copy()
# X_train_attack['model_confidence'] = train_confidences

# X_test_attack = X_test[attack_features].copy()
# X_test_attack['model_confidence'] = test_confidences


# y_train_attack = X_train[sensitive_feature]
# y_test_attack = X_test[sensitive_feature]

# # Train the Inference Model (using deep learning)
# rf_model.fit(X_train_attack, y_train_attack)

# # Evaluate the Inference Model
# y_pred_attack = rf_model.predict(X_test_attack)

# attack_accuracy = accuracy_score(y_test_attack, y_pred_attack)

# train_mean_conf = np.mean(train_confidences)
# test_mean_conf = np.mean(test_confidences)

# print("\nAttribute Inference Attack on 'Income':")
# print(f"Attack Accuracy: {attack_accuracy:.2f}")
# print(f"Avg Train Confidence: {train_mean_conf:.2f}")
# print(f"Avg Test Confidence: {test_mean_conf:.2f}\n")

# report = classification_report(y_test_attack, y_pred_attack, output_dict=True)

# def print_classification_report(report):
#     headers = ["Metric"] + [str(label) for label in report if label not in ('accuracy', 'macro avg', 'weighted avg')]
#     rows = [
#         ["Precision"] + [report[label]['precision'] for label in report if label not in ('accuracy', 'macro avg', 'weighted avg')],
#         ["Recall"] + [report[label]['recall'] for label in report if label not in ('accuracy', 'macro avg', 'weighted avg')],
#         ["F1-Score"] + [report[label]['f1-score'] for label in report if label not in ('accuracy', 'macro avg', 'weighted avg')],
#         ["Support"] + [report[label]['support'] for label in report if label not in ('accuracy', 'macro avg', 'weighted avg')]
#     ]

#     # Add macro and weighted averages
#     rows.append(["Macro Avg", report['macro avg']['precision'], report['macro avg']['recall'], report['macro avg']['f1-score'], report['macro avg']['support']])
#     rows.append(["Weighted Avg", report['weighted avg']['precision'], report['weighted avg']['recall'], report['weighted avg']['f1-score'], report['weighted avg']['support']])
#     rows.append(["Accuracy", "", "", report['accuracy'], ""])

#     print(tabulate(rows, headers=headers, tablefmt="grid", floatfmt=".2f"))

# plt.figure(figsize=(10, 7))
# plt.hist(train_confidences, bins=50, alpha=0.6, label="Train (Member)", color='blue')
# plt.hist(test_confidences, bins=50, alpha=0.6, label="Test (Non-Member)", color='red')
# plt.axvline(train_mean_conf, color='blue', linestyle='dashed', linewidth=1, label=f"Mean Train Confidence: {train_mean_conf:.2f}")
# plt.axvline(test_mean_conf, color='red', linestyle='dashed', linewidth=1, label=f"Mean Test Confidence: {test_mean_conf:.2f}")
# plt.xlabel("Model Confidence Score")
# plt.ylabel("Count")
# plt.legend()
# plt.title("Attribute Inference Attack - Confidence Distribution")
# plt.show()


The attack achieved 47% accuracy, meaning it correctly inferred the income level of individuals nearly half the time. As the Income attribute has 8 levels, it is normal that the accuracy is lower. If the attack tried to guess randomly, it would be around 12,5% (1/8 *100). Therefore data is leaking. 

Also, the model showed higher confidence on training data (0.91) than test data (0.83), indicating a potential privacy risk.

<!-- #### 6.4 Find most exposed individuals and the analyzing the infered data -->

In [ ]:
# # Add predictions and confidence to the test dataset for analysis
# vulnerable_data = X_test_attack.copy()
# vulnerable_data['True_Income'] = y_test_attack.values
# vulnerable_data['Predicted_Income'] = y_pred_attack

# # Calculate prediction correctness
# vulnerable_data['Correct_Prediction'] = (vulnerable_data['True_Income'] == vulnerable_data['Predicted_Income'])

# # Fidn top 20 most vulnerable individuals (highest confidence)
# most_vulnerable = vulnerable_data.sort_values(by='model_confidence', ascending=False).head(20)

# print("Most Vulnerable Individuals (Top 20 by Confidence Score):")
# print(tabulate(most_vulnerable[['model_confidence', 'True_Income', 'Predicted_Income', 'Correct_Prediction']],
#                headers='keys', tablefmt='pretty'))

The 20 most vulnerable individuals had a model confidence of 1.0, meaning the model was completely certain about their predicted income. 16 out of 20 predictions were completely correct (80%), highlighting a significant privacy concern for these individuals.

## 7 . Evaluation & Performance Metrics

Missing

## 8 . Results & Discussion

Missing

## 9 . Conclusion & Future Work

Missing